<h1>Extracting and Visualizing Stock Data</h1>


<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li>Defining function that Makes a Graph</li>
        <li>Using yfinance to Extract GME's Stock Data</li>
        <li>Analyzing GameStop Stock</li>
    </ul>

  Reference : https://www.kaggle.com/code/faressayah/stock-market-analysis-prediction-using-lstm

</div>

<hr>


In [12]:
#installing required libraries
!pip install yfinance
!pip install pandas
!pip install --upgrade nbformat
!pip install plotly

In [29]:
#dependencies
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

Extracting GME'**s** Stock Data

In [31]:
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")

gme_data = gme_data.reset_index()
#order by date
gme_data.sort_values(by='Date' , ascending=True , inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712708,1.716074,1.670627,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


Analysing GME's Stock data

In [32]:
#Data Available

print("Number of days for which stock data is available" , gme_data.shape[0], '\n')
print('Range of days for which the stock data is available' , gme_data.Date.min() , 'to' , gme_data.Date.max() , '\n')
print('Information about dataset' , gme_data.info(), '\n')
print('Summary of the dataset:' , gme_data.describe(), '\n')

#Information about dataset
#Date : date when the stock price is given
#Open: Opening trading stock price for the given date
#High: Highest stock price traded for the given date
#Low: Lowest trading stock price traded for the given date
#Close: Closing trading stock price for the given date
#Volume : Traded number of stocks for the given date
#Dividends: Stock dividend is a distribution of additional shares of a company's stock to existing shareholders, https://www.investopedia.com/terms/d/dividendrate.asp
#Stock Splits: Stock split is done to divide the existing shares into multiple shares.

Number of days for which stock data is available 5408 

Range of days for which the stock data is available 2002-02-13 00:00:00-05:00 to 2023-08-08 00:00:00-04:00 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5408 entries, 0 to 5407
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype                           
---  ------        --------------  -----                           
 0   Date          5408 non-null   datetime64[ns, America/New_York]
 1   Open          5408 non-null   float64                         
 2   High          5408 non-null   float64                         
 3   Low           5408 non-null   float64                         
 4   Close         5408 non-null   float64                         
 5   Volume        5408 non-null   int64                           
 6   Dividends     5408 non-null   float64                         
 7   Stock Splits  5408 non-null   float64                         
dtypes: datetime64[ns, America/New_York](1), floa

### Closing Price
The closing price is the last price at which the stock is traded during the regular trading day. A stock’s closing price is the standard benchmark used by investors to track its performance over time.

In [33]:
#Analysing the different prices

def prices_plot(stock_data):
  fig = px.line(stock_data, x="Date", y=['Open' , 'Close' , 'Low' , 'High'],
              hover_data={"Date": "|%B %d, %Y"},
              title='Stock Price Plot')
  fig.update_xaxes(
      dtick="M1",
      tickformat="%b\n%Y")
  fig.show()

prices_plot(gme_data)

#Observation : Around 28th Jan, 2021 the high stock price shoot up to 120.75$
#Low Stock price : 3 peaks - Jan 27th, 2021 - 62.25$ , March 12 , 2021 - 65.56$, June 9th 2021 - 72.8775$
#Closing Stock price : Jan 27th , 2021 - 86.8775$
#Opening Stock price : Jan 29th , 2021 - 94.9275$

### Trading Volume
Volume is the amount of an asset or security that changes hands over some period of time, often over the course of a day. For instance, the stock trading volume would refer to the number of shares of security traded between its daily open and close. </br>
Trading volume, and changes to volume over the course of time, are important inputs for technical traders.

In [34]:
def trading_volume_plot(stock_data):
  fig = px.line(stock_data, x="Date", y='Volume',
              hover_data={"Date": "|%B %d, %Y"},
              title='Trading Volume of GME stock')
  fig.update_xaxes(
      dtick="M1",
      tickformat="%b\n%Y")
  fig.show()

trading_volume_plot(gme_data)

#observation: On 22dn Jan 2021, 788 M stock trades happened during the day

### Trend of the stock price using Moving Averages

The moving average (MA) is a simple technical analysis tool that smooths out price data by creating a constantly updated average price. The average is taken over a specific period of time, like 10 days, 20 minutes, 30 weeks, or any time period the trader chooses.

In [40]:
gme_data['moving_average_10days'] = gme_data['Close'].rolling(10).mean()
gme_data['moving_average_20days'] = gme_data['Close'].rolling(20).mean()
gme_data['moving_average_50days'] = gme_data['Close'].rolling(50).mean()

fig = make_subplots(rows=1, cols=3 , subplot_titles=("Stock Price Moving Average over 10 days", \
                                                     "Stock Price Moving Average over 20 days", \
                                                     "Stock Price Moving Average over 50 days"))

fig.add_trace(
    go.Scatter(x=gme_data['Date'], y=gme_data['moving_average_10days'] , \
               name="Stock Price Moving Average over 10 days"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=gme_data['Date'], y=gme_data['moving_average_20days'] , \
               name="Stock Price Moving Average over 20 days"),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=gme_data['Date'], y=gme_data['moving_average_50days'] , \
               name="Stock Price Moving Average over 50 days"),
    row=1, col=3
)

fig.update_layout(height=400, width=1500, title_text="Moving Average Subplots")
fig.show()

#Observation: 50 days moving average smotthen the price out pretty well and shows the trend of the closing stock price which increased upto July 2021 and then reduced.

### Daily returns of the stock

This is given by percentage change of closing price from previous day : (closing price on particular day - closing price on previous day)/closing price on previous day.

In [43]:
#ensure that the data is sorted in ascending order by date
gme_data['pct_change_closing_price'] = gme_data['Close'].pct_change()

fig = px.line(gme_data, x="Date", y='pct_change_closing_price',
              hover_data={"Date": "|%B %d, %Y"},
              title='Daily return of GME stock')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

fig.show()

#observation : Return (% change in closing price was highest on Jan 27th 2021 i.e the stock price dastrically increased by more than 100% -> 134.8356% compared to previous day)

### Variability in the trading price during the day

If the high and low trading stock price differ a lot during the trading day, it means that there is more variability in the stock price and hence more risk in purchasing/selling the stock.

In [44]:
gme_data['variability_high_low_price'] = (gme_data.High - gme_data.Low)*100/gme_data.Low

fig = px.line(gme_data, x="Date", y='variability_high_low_price',
              hover_data={"Date": "|%B %d, %Y"},
              title='Daily return variability/risk of GME stock')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

fig.show()

#observation: the graph shows that the variability in the stock price during the day was very high during Jan 2021 period.

## How frequently does the company pay dividends to its shareholders?

If a firm pays dividends to the shareholders regularly, it is an indicator that the firm is working well.

Reference: https://www.investopedia.com/terms/d/dividendrate.asp

## How frequently does the firm do stock splits

It is usually done to increase the liquidity of the trading shares. A company often decides on a split when the stock price is quite high, making it expensive for investors to acquire a standard board lot of 100 shares. A stock split may be viewed by some as a company wanting a bigger future runway for growth; for this reason, a stock split generally indicates executive-level confidence in the prospect of a company.

Reference: https://www.investopedia.com/terms/s/stocksplit.asp

In [46]:
fig = px.line(gme_data, x="Date", y=['Dividends', 'Stock Splits'],
              hover_data={"Date": "|%B %d, %Y"},
              title='Dividends/Stock Splits of GME stock')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

fig.show()

#Observation : Shows that the firm paid dividends 2012 to 2019 but didn't pay anything afterwards
#Observation: Shows that the firm did the stock splits twice during the time period

In [47]:
#did share price change after the stock split?

fig = px.line(gme_data[(gme_data.Date >= '2022-07-22') & (gme_data.Date <= '2022-08-30')], x="Date", y='Close',
              hover_data={"Date": "|%B %d, %Y"},
              title='Closing Price of GME stock after stock split on 22nd july 2022')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

fig.show()

#Observation: The stock price reduces in the short terms and then increases
